# Classifying MNIST Digits with a Neural Network Made From Scratch

This notebook demonstrates how to build and train a simple neural network from scratch. The overall process involves initializing network parameters, performing a forward pass to obtain predictions, computing the loss, and using backpropagation to update the parameters. Below is an overview of the main components.

This notebook is heavily inspired by [Samson Zhang](https://www.youtube.com/watch?v=w8yWXqWQYmU).

## Network Architecture

We construct our network with an input layer, one or more hidden layers, and an output layer. For each layer \(l\), the weighted sum (also known as the pre-activation output) is computed by:

$$
z^{(l)} = W^{(l)} a^{(l-1)} + b^{(l)}
$$

- **$\(W^{(l)}\$)**: Weight matrix for layer \(l\).

- **$\(a^{(l-1)}\$)**: Activation from the previous layer.

- **$\(b^{(l)}\$)**: Bias term for layer \(l\).

Here, $(z^{(l)}$) is then passed through an activation function to produce the activation $(a^{(l)}$) for that layer.


## 2. Activation Functions

Activation functions introduce non-linearities into the network, which enable it to capture complex relationships. Some common choices are:

### Sigmoid
The sigmoid function maps input values to a range between 0 and 1. Its formula is:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

This function is especially useful in the output layer of binary classifiers.

### Rectified Linear Unit (ReLU)
ReLU is defined as:

$$
\text{ReLU}(z) = \max(0, z)
$$

It is popular because it mitigates the vanishing gradient problem during training.

## 3. Loss Function Explanation

While this notebook does not explicitly compute a scalar loss value, it uses the gradient of the loss to update parameters. The key idea is that, when combining softmax with the cross-entropy loss, the derivative with respect to the pre-softmax activations $( Z_2 $) simplifies to:

$$
dZ_2 = A_2 - Y_{\text{one-hot}}
$$

Here, $( Y_{\text{one-hot}} $) is the one-hot encoded vector of the true label. The corresponding cross-entropy loss is defined as:

$$
L(Y, A_2) = -\sum_{i} Y_i \log(A_{2,i})
$$

Because of the properties of softmax, its derivative together with cross-entropy yields the simple error term $( A_2 - Y_{\text{one-hot}} $).

## 4. Backpropagation and Parameter Updates

Backpropagation is the key algorithm for training neural networks. It involves computing gradients of the loss function with respect to each parameter. These gradients are then used to update the parameters via gradient descent. The generic update rule is:

$$
\theta := \theta - \alpha \frac{\partial L}{\partial \theta}
$$

Where:
- $\( \theta \$) represents any network parameter (e.g., \(W\) or \(b\)).

- $\( \alpha \$) is the learning rate, controlling the size of the update step.

- $\( \frac{\partial L}{\partial \theta} \$) is the gradient of the loss with respect to the parameter.

Gradient computation is carried out using the chain rule.

For a weight matrix $W^{(l)}$ in layer $l$:

$$
\frac{\partial L}{\partial W^{(l)}} = \frac{\partial L}{\partial z^{(l)}} \cdot \frac{\partial z^{(l)}}{\partial W^{(l)}}
$$

This derivative tells us how changes in $\(W^{(l)}\$) affect the loss, allowing the algorithm to minimize the loss function iteratively.

## Problem Setup

In [92]:
# Package imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [93]:
### MNIST
data = pd.read_csv('/content/mnist_train.csv')

## Data Formatting

In [94]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [95]:
Y_train.shape

(59000,)

In [96]:
X_train.shape

(784, 59000)

## Neural Net Functions

In [97]:
def init_params():
    '''
    Initializes the weights and biases matrices for the neural network.

    np.random.rand() by default provides [0, 1). However, here we subtract by
    0.5 to make that [-0.5, 0.5).

    Returns initializes weights & biases.
    '''
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5

    return W1, b1, W2, b2

def ReLU(Z):
    '''
    Calculates the ReLU output for a value.
    0 if <= 0.
    Z if > 0.
    '''
    return np.maximum(Z, 0)

def softmax(Z):
    '''
    Calculates the softmax function on a group of logits. The resulting output
    is a vector of probabilities, the highest of which will be the label that
    the model predicts.
    '''

    # Subtract the max for numerical stability
    Z_shifted = Z - np.max(Z, axis=0, keepdims=True)

    # Calculate probabilities. Here we look at each part with respect to the
    # whole, which provides us a ratio (probability)
    expZ = np.exp(Z_shifted)

    # We add an eps here for numerical stability with potential edge cases
    A = expZ / (np.sum(expZ, axis=0, keepdims=True) + 1e-5)

    return A

def forward_prop(W1, b1, W2, b2, X):
    '''
    Performs forward propagation, which is the process of feeding data
    through the network and getting a prediction, which we can then measure the
    error on via backprop.

    Returns intermediate and final value from forward prop operation.
    '''

    Z1 = W1.dot(X) + b1 # Output of layer 1 (pre-activation)
    A1 = ReLU(Z1) # Activation applied to layer 1
    Z2 = W2.dot(A1) + b2 # Output of layer 2
    A2 = softmax(Z2) # Classification output

    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    '''
    Calculates the derivative of the ReLU function.

    ReLU is defined as:
      x if x > 0
      0 if x <= 0

    Therefore, we can return a Boolean of True (1) if the value is positive, 0
    otherwise, which equates to the slope (gradient) of the function at that
    point.
    '''

    return Z > 0

def one_hot(Y):
    '''
    One-hot encodes the outcome label. This ensures that the outcome label
    is of the same shape as the number of labels, but we add a column component
    that has a 1 in the slot of the correct label (here, 0 - 9) and zeroes
    everywhere else.

    Returns one-hot encoded array.
    '''
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) # (num_examples, 9 + 1)
    one_hot_Y[np.arange(Y.size), Y] = 1 # Indexes array of zeroes and places 1
                                        # at label position

    one_hot_Y = one_hot_Y.T # Transposes matrix for mat mul

    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
  '''
  Perform backpropagation using chain rule.
  Takes weights, biases, output for layer 1 and two,
  as well as the activation functions performed on said layers.

  Uses the number of training examples, m, for derivative calcs.

  For derivative of biases for layers 1 and two, I use keepdims
  to ensure that the resulting gradient has the same shape as b2 and b1.

  Returns gradients for weights and biases for both layers in network.
  '''

  m = X.shape[1]  # Number of training examples
  one_hot_Y = one_hot(Y) # One-hot encode label vector

  dZ2 = A2 - one_hot_Y # Gradient of the output w/ respect to the true label
  dW2 = 1 / m * dZ2.dot(A1.T) # Gradient of the weight matrix wrt the layer 2
  db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True) # Gradient of the bias wrt layer 2
  dZ1 = W2.T.dot(dZ2) * (Z1 > 0) # gradient of layer one output wrt activation
                                 # of layer one and the impact of the layer 2 weight matrix
                                 # on the layer 2 output.

  dW1 = 1 / m * dZ1.dot(X.T)  # gradient of the layer 1 weight matrix wrt the layer 1
                              # output and input

  db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True) # gradient of layer 1 bias
                                                   # wrt layer 1 output

  return dW1, db1, dW2, db2


def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    '''
    Performs gradient descent step. Here, we modify the weights and biases
    based upon the errors calculated via backprop.

    The weights are adjusted based upon the learning rate to help ensure
    smooth updates across training.
    '''
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2

    return W1, b1, W2, b2

In [98]:
def get_predictions(A2):
    '''
    Returns the highest output from the softmax function.
    '''
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    '''
    Calculates accuracy of the model predictions.

    Also showcases model preds alongside true labels.
    '''
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    '''
    Performs gradient descent.

    First, it grabs initial values for weight & bias matrices.
    Then, for how many specified iterations, it performs forward prop, then back
    prop, then a weight and bias update.

    Every ten iterations, it will show current model accuracy on its forward
    prop run.

    Returns updated params for weight & bias matrices.
    '''
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

## Perform Gradient Descent

In [99]:
# Perform gradient descent / train model
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[9 9 9 ... 9 9 9] [1 5 7 ... 9 1 7]
0.10008474576271187
Iteration:  10
[6 7 0 ... 8 2 7] [1 5 7 ... 9 1 7]
0.1450677966101695
Iteration:  20
[8 7 0 ... 8 8 7] [1 5 7 ... 9 1 7]
0.2086271186440678
Iteration:  30
[1 7 0 ... 8 8 7] [1 5 7 ... 9 1 7]
0.27177966101694917
Iteration:  40
[1 0 0 ... 8 1 7] [1 5 7 ... 9 1 7]
0.33559322033898303
Iteration:  50
[1 0 0 ... 9 1 7] [1 5 7 ... 9 1 7]
0.39961016949152545
Iteration:  60
[1 0 0 ... 9 1 5] [1 5 7 ... 9 1 7]
0.45796610169491525
Iteration:  70
[1 0 0 ... 9 1 5] [1 5 7 ... 9 1 7]
0.513542372881356
Iteration:  80
[1 5 0 ... 9 1 8] [1 5 7 ... 9 1 7]
0.5601525423728814
Iteration:  90
[1 5 7 ... 9 1 8] [1 5 7 ... 9 1 7]
0.6003728813559323
Iteration:  100
[1 5 7 ... 9 1 7] [1 5 7 ... 9 1 7]
0.6331525423728813
Iteration:  110
[1 5 7 ... 9 1 7] [1 5 7 ... 9 1 7]
0.6596440677966102
Iteration:  120
[1 5 7 ... 9 1 7] [1 5 7 ... 9 1 7]
0.6835593220338984
Iteration:  130
[1 5 7 ... 9 1 7] [1 5 7 ... 9 1 7]
0.7044915254237288
Iteration:  1

## Test Dataset

In [100]:
def make_predictions(X, W1, b1, W2, b2):
    '''
    Executes forward propagation and gets predictions from the model.
    '''
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def get_accuracy_test(predictions, Y):
    '''
    A variant of the previous accuracy test. However, here we do not print out
    any of the predictions. We just store the accuracy value.
    '''
    return np.sum(predictions == Y) / Y.size

In [101]:
# Run predictions on test set
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)

# Output accuracy on test set
print(
    f'Test set accuracy: {float(get_accuracy_test(dev_predictions, Y_dev)*100)}%'
    )

Test set accuracy: 86.4%
